# Pokémon Type Predictor using K-Means Clustering
This notebook builds a fun Gradio app that lets users enter Pokémon stats and predicts the most likely Pokémon Type using K-Means clustering (an unsupervised learning algorithm).

In [ ]:
# Install required libraries (uncomment if not already installed)
# !pip install pandas scikit-learn gradio matplotlib


## Step 1: Load and Explore the Dataset

In [ ]:
import pandas as pd

# Load dataset (make sure the CSV is in your working directory)
df = pd.read_csv('Pokemon.csv')
df = df[['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed', 'Type 1']]
df.dropna(inplace=True)
df.head()

## Step 2: Preprocess the Data

In [ ]:
from sklearn.preprocessing import StandardScaler

# Features for clustering
X = df.drop(columns=['Type 1'])
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## Step 3: Apply K-Means Clustering

In [ ]:
from sklearn.cluster import KMeans

# Using 18 clusters since there are roughly 18 Pokemon Types
kmeans = KMeans(n_clusters=18, random_state=42)
clusters = kmeans.fit_predict(X_scaled)
df['Cluster'] = clusters

## Step 4: Map Clusters to Most Frequent Type in Each Cluster

In [ ]:
# Map cluster number to most common type in that cluster
type_map = df.groupby('Cluster')['Type 1'].agg(lambda x: x.value_counts().index[0])
cluster_to_type = dict(type_map)
cluster_to_type

## Step 5: Build Gradio Interface

In [ ]:
import gradio as gr

def predict_type(hp, attack, defense, sp_atk, sp_def, speed):
    user_input = pd.DataFrame([[hp, attack, defense, sp_atk, sp_def, speed]],
                              columns=['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed'])
    user_scaled = scaler.transform(user_input)
    cluster = kmeans.predict(user_scaled)[0]
    return f"Predicted Pokémon Type: {cluster_to_type[cluster]}"

interface = gr.Interface(
    fn=predict_type,
    inputs=[
        gr.Slider(1, 255, label="HP"),
        gr.Slider(5, 190, label="Attack"),
        gr.Slider(5, 230, label="Defense"),
        gr.Slider(10, 194, label="Sp. Atk"),
        gr.Slider(20, 230, label="Sp. Def"),
        gr.Slider(5, 200, label="Speed")
    ],
    outputs="text",
    title="Pokémon Type Predictor",
    description="Enter Pokémon stats and get a predicted type using K-Means Clustering!"
)

# Uncomment to launch the app
# interface.launch()